In [96]:
# Reading in the initial data

import pandas as pd

matches = pd.read_csv("matches.csv", index_col=0)

In [97]:
## Exploring Data Set
print(matches.head())

         date   time            comp        round  day venue result   gf   ga  \
1  2021-08-15  16:30  Premier League  Matchweek 1  Sun  Away      L  0.0  1.0   
2  2021-08-21  15:00  Premier League  Matchweek 2  Sat  Home      W  5.0  0.0   
3  2021-08-28  12:30  Premier League  Matchweek 3  Sat  Home      W  5.0  0.0   
4  2021-09-11  15:00  Premier League  Matchweek 4  Sat  Away      W  1.0  0.0   
6  2021-09-18  15:00  Premier League  Matchweek 5  Sat  Home      D  0.0  0.0   

         opponent  ...  match report  notes    sh   sot  dist   fk   pk pkatt  \
1       Tottenham  ...  Match Report    NaN  18.0   4.0  16.9  1.0  0.0   0.0   
2    Norwich City  ...  Match Report    NaN  16.0   4.0  17.3  1.0  0.0   0.0   
3         Arsenal  ...  Match Report    NaN  25.0  10.0  14.3  0.0  0.0   0.0   
4  Leicester City  ...  Match Report    NaN  25.0   8.0  14.0  0.0  0.0   0.0   
6     Southampton  ...  Match Report    NaN  16.0   1.0  15.7  1.0  0.0   0.0   

   season             team

In [98]:
print(matches.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1389 entries, 1 to 42
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1389 non-null   object 
 1   time          1389 non-null   object 
 2   comp          1389 non-null   object 
 3   round         1389 non-null   object 
 4   day           1389 non-null   object 
 5   venue         1389 non-null   object 
 6   result        1389 non-null   object 
 7   gf            1389 non-null   float64
 8   ga            1389 non-null   float64
 9   opponent      1389 non-null   object 
 10  xg            1389 non-null   float64
 11  xga           1389 non-null   float64
 12  poss          1389 non-null   float64
 13  attendance    693 non-null    float64
 14  captain       1389 non-null   object 
 15  formation     1389 non-null   object 
 16  referee       1389 non-null   object 
 17  match report  1389 non-null   object 
 18  notes         0 non-null      

In [99]:
## Looking at different viewpoints for the data set allows me to see what I'm working with more clearly.
## I can start to pick out what I think will be the most important stats to determine winners
## This includes goals for, goals against, possession, shots and shots on target

print(matches.describe())

                gf           ga           xg          xga         poss  \
count  1389.000000  1389.000000  1389.000000  1389.000000  1389.000000   
mean      1.335493     1.380850     1.304176     1.338445    49.702664   
std       1.274235     1.291049     0.767268     0.789360    12.401897   
min       0.000000     0.000000     0.000000     0.000000    18.000000   
25%       0.000000     0.000000     0.700000     0.700000    40.000000   
50%       1.000000     1.000000     1.200000     1.200000    50.000000   
75%       2.000000     2.000000     1.800000     1.800000    59.000000   
max       9.000000     9.000000     4.600000     5.000000    82.000000   

         attendance  notes           sh          sot         dist  \
count    693.000000    0.0  1389.000000  1389.000000  1388.000000   
mean   36089.963925    NaN    12.153348     4.041037    17.011527   
std    17797.991778    NaN     5.268876     2.403866     2.988364   
min     2000.000000    NaN     0.000000     0.000000     

In [100]:
## Viewing all columns in the data set
print(matches.columns)

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team'],
      dtype='object')


In [101]:
## By sorting by matches per team, I can see that Liverpool has less data than expected.

matches['team'].value_counts()

Southampton                 72
Manchester United           72
Newcastle United            72
Brighton and Hove Albion    72
West Ham United             72
Tottenham Hotspur           71
Arsenal                     71
Burnley                     71
Leeds United                71
Crystal Palace              71
Wolverhampton Wanderers     71
Manchester City             71
Everton                     70
Chelsea                     70
Aston Villa                 70
Leicester City              70
Fulham                      38
Sheffield United            38
West Bromwich Albion        38
Liverpool                   38
Brentford                   34
Norwich City                33
Watford                     33
Name: team, dtype: int64

In [102]:
## Dropping cells that won't be useful

del matches['comp']
del matches['notes']

In [103]:
## Converting the Date to a useable format

matches["date"] = pd.to_datetime(matches["date"])

In [104]:
## Converting Wins and Losses into numeric values
## Win = 1
## This target column is what we'll use to compare our model for accuracy
## Note: Currently, a draw is considered a loss.

matches["target"] = (matches["result"] == "W").astype("int")

In [105]:
## Adding column to signify a home game as 1

matches["home/away"] = (matches["venue"] == "Home").astype("int")

In [106]:
## Creating columns to numerically represent opponents and venue for use in predictions

matches["opp_code"] = (matches["opponent"]).astype("category").cat.codes
matches["venue_code"] = (matches["venue"]).astype("category").cat.codes

In [107]:
## Removing the minutes and ':' from the time column to be able to easily utilize the data

matches['hour'] = matches['time'].str.replace(':.+','', regex=True).astype('int')

In [108]:
## Adding a column to numerically signify day of week

matches["day_code"] = matches["date"].dt.dayofweek

## Looking at the matches dataframe below, we can see that the default for pandas uses 6 for sunday, 5 for saturday, etc..

In [109]:
matches.head()

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,pk,pkatt,season,team,target,home/away,opp_code,venue_code,hour,day_code
1,2021-08-15,16:30,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,1.9,...,0.0,0.0,2022,Manchester City,0,0,18,0,16,6
2,2021-08-21,15:00,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,2.7,...,0.0,0.0,2022,Manchester City,1,1,15,1,15,5
3,2021-08-28,12:30,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,3.8,...,0.0,0.0,2022,Manchester City,1,1,0,1,12,5
4,2021-09-11,15:00,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,2.9,...,0.0,0.0,2022,Manchester City,1,0,10,0,15,5
6,2021-09-18,15:00,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,1.1,...,0.0,0.0,2022,Manchester City,0,1,17,1,15,5


#### Thought Process: Factors that would be useful in predicting a match winner would include:
##### Home Team, Shots, Shots on Target, Possession, Expected Goals (xg), and Expected Goals against (xga)
##### Since some of these factors won't be known until after the game is over (shots, possession, etc) a function will have to be created to take the average from previous matchweeks.

In [110]:
## Making sure all the data I want to use is in a suitable data type (float, int) for machine learning

matches.dtypes

date            datetime64[ns]
time                    object
round                   object
day                     object
venue                   object
result                  object
gf                     float64
ga                     float64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                     float64
pkatt                  float64
season                   int64
team                    object
target                   int64
home/away                int64
opp_code                  int8
venue_code                int8
hour                     int64
day_code                 int64
dtype: object

In [111]:
## Random Forest Classifier is able to recognize the opponent codes correspond to teams instead of being linear
from sklearn.ensemble import RandomForestClassifier

In [112]:
##n_estimators can be increased later for more accuracy, initially it will be set small for processing speed

rf = RandomForestClassifier(n_estimators=50,min_samples_split=10, random_state = 1)

In [113]:
## For the training model we will use matches from 2021
train = matches[matches['date'] < '2022-01-01']

In [114]:
## For testing, we will use matches in 2022
test = matches[matches['date'] > '2022-01-01']

In [115]:
predictors = ['venue_code', 'opp_code', 'hour', 'day_code']

In [116]:
rf.fit(train[predictors], train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [117]:
preds = rf.predict(test[predictors])

In [118]:
## Use this to find % of time the predictions are accuracte
from sklearn.metrics import accuracy_score

In [119]:
acc = accuracy_score(test['target'], preds)

In [120]:
acc

0.6123188405797102

#### In our initial test, we predicted about 61% of matches correctly. We can take a closer look to see where we can improve

In [121]:
combined = pd.DataFrame(dict(actual=test['target'], prediction = preds))

In [122]:
pd.crosstab(index=combined['actual'], columns =combined['prediction'])

prediction,0,1
actual,,
0,141,31
1,76,28


#### Predicting Losses/Draws were more accurate than predicting wins

In [123]:
from sklearn.metrics import precision_score

In [124]:
## When prediction a win, this model only got it right 48% of the time

precision_score(test['target'], preds)

0.4745762711864407

### To be able to improve the accuracy, I need more predictors. I can use stats that I previously marked as important by creating averages per team for matches that have been previously played

In [125]:
## Grouping matches by team
grouped_matches = matches.groupby('team')

In [126]:
group = grouped_matches.get_group('Manchester City')

In [127]:
group.head()

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,pk,pkatt,season,team,target,home/away,opp_code,venue_code,hour,day_code
1,2021-08-15,16:30,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,1.9,...,0.0,0.0,2022,Manchester City,0,0,18,0,16,6
2,2021-08-21,15:00,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,2.7,...,0.0,0.0,2022,Manchester City,1,1,15,1,15,5
3,2021-08-28,12:30,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,3.8,...,0.0,0.0,2022,Manchester City,1,1,0,1,12,5
4,2021-09-11,15:00,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,2.9,...,0.0,0.0,2022,Manchester City,1,0,10,0,15,5
6,2021-09-18,15:00,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,1.1,...,0.0,0.0,2022,Manchester City,0,1,17,1,15,5


In [128]:
## Defining a function to compute averages of previous matches to use in predictions
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [129]:
## Creating new columns for average values I want for predictions
cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']
new_cols = [f'{c}_rolling' for c in cols]

In [130]:
new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [133]:
rolling_averages(group, cols, new_cols)

ValueError: closed only implemented for datetimelike and offset based windows